洪泛算法：在一个连通网络中，每个进程$p_i$持有一个初始值$v_i$，目标是让所有进程都输出全局最大值$\max_{i}v_i$。每个进程只在自己当前最大值发生变化时，才把新的最大值发送给邻居

系统模型：网络为无向连通图，进程为同步或异步，通信可靠无损，每个进程只知道自己的邻居。

时间复杂度：$O(D)$ $\ \ \ \ \ $ 消息复杂度$O(E\cdot U)$


In [5]:
from collections import deque
class Node:
    def __init__(self, node_id, initial_value):
        self.id=node_id
        self.initial_value=initial_value
        self.max_val=initial_value
        self.neighbors=[]

    def __repr__(self):
        return f"Node({self.id}, max={self.max_val})"

class OptFloodMaxSimulator:
    def __init__(self, graph, values):
        """
        graph: dict {node_id: [neighbor_ids]}
        values: dict {node_id: initial_value}
        """
        self.nodes={
            i: Node(i, values[i]) for i in graph
        }
        for i, nbrs in graph.items():
            self.nodes[i].neighbors=nbrs
        self.message_queue=deque()
        self.message_count=0

    def send(self, sender, receiver, value):
        self.message_queue.append((sender, receiver, value))
        self.message_count+=1

    def initialize(self):
        for node in self.nodes.values():
            for nbr in node.neighbors:
                self.send(node.id, nbr, node.max_val)

    def run(self, verbose=True):
        self.initialize()
        while self.message_queue:
            sender, receiver, value = self.message_queue.popleft()
            recv_node = self.nodes[receiver]
            if verbose:
                print(f"Message {sender} -> {receiver}: {value}")
            if value > recv_node.max_val:
                old = recv_node.max_val
                recv_node.max_val=value
                if verbose:
                    print(
                        f"  Node {receiver} updates max"
                        f"{old} -> {value}"
                    )
                for nbr in recv_node.neighbors:
                    if nbr!=sender:
                        self.send(receiver, nbr, value)
        if verbose:
            print("\nAlgorithm converged.")
            print(f"Total messages sent: {self.message_count}")

    def result(self):
        return{
            node_id: node.max_val
            for node_id, node in self.nodes.items()
        }

if __name__ =="__main__":
    graph = {
        1: [2, 3],
        2: [1, 4],
        3: [1, 4],
        4: [2, 3]
    }

    values = {
        1: 5,
        2: 12,
        3: 7,
        4: 9
    }

    sim = OptFloodMaxSimulator(graph, values)
    sim.run(verbose=True)

    print("\nFinal result:")
    print(sim.result())


Message 1 -> 2: 5
Message 1 -> 3: 5
Message 2 -> 1: 12
  Node 1 updates max5 -> 12
Message 2 -> 4: 12
  Node 4 updates max9 -> 12
Message 3 -> 1: 7
Message 3 -> 4: 7
Message 4 -> 2: 9
Message 4 -> 3: 9
  Node 3 updates max7 -> 9
Message 1 -> 3: 12
  Node 3 updates max9 -> 12
Message 4 -> 3: 12
Message 3 -> 1: 9
Message 3 -> 4: 12

Algorithm converged.
Total messages sent: 12

Final result:
{1: 12, 2: 12, 3: 12, 4: 12}


SynchBFS算法：给定一个连通网络和一个指定根节点，构造一棵BFS树，树中每个节点到根的距离等于最短路径长度，每个节点确定父节点和到根的层数

模型假设：同步消息传递模型
1. 全局轮次：所有进程在统一论发送消息、接收消息、更新本地状态
2. 可靠通信：无丢包，无伪造
3. 无向连通图
4. 每个节点只知道自己的邻居

算法流程：
1. 第0轮：初始化
2. 第r轮：每个节点p在本轮接受完消息后执行：

$\ \ \ \ $若收到任意消息<LEVEL, r-1>且当前level_p=∞，则level_p:=r, parent_p:= 发送该消息的邻居，向所有邻居发送<LEVEL, r>。否则不做任何事

SynchGHS算法：在不知道全局图的情况下，通过不断合并“片段”，每个片段在每一阶段选择其“最小外向边”，并在同步轮次中合并，最终得到MST

片段：MST的一个部分树，每个节点属于且仅属于一个片段

片段等级：表示片段规模的指数级标记，等级单调递增，用于避免冲突与循环

最小外向边：一条连接片段内节点与片段外节点的，权值最小的边

算法流程：
1. Phase0：初始化，每个节点是一个独立片段，level=0，fragment_id=自身ID；所有边为Basic
2. Phase1：寻找MOE：

$\ \ \ \ \ $ 1.TEST：节点向 Basic 边的邻居发送 TEST

$\ \ \ \ \ $ 2.ACCEPT/REJECT:若对方片段不同 → ACCEPT；否则REJECT

$\ \ \ \ \ $ 3.REPORT：子节点向父节点汇报找到的最小外向边，根节点汇总得到整个片段的 MOE

3. Phase2：合并阶段：根节点在MOE上发起CONNECT，低等级->高等级，同等级->合并且等级+1；合并后通过INITIATE向下广播新片段消息。保证了无环，无死锁

4. Phase3：重复直到只剩一个片段：当前所有节点属于同一片段，且所有边非Basic，算法终止，Branch边集合即MST

时间复杂度：$O(nlogn)$ $\ \ \ \ \ $ 消息复杂度：$O(E+nlogn)$

LubyMIS算法：每一轮中，每个节点随机竞争；若它的随机值严格大于所有邻居，则胜出进入MIS，并删除在自己及邻居，重复如上过程直到结束。引入了随机化思想来破除对称性

前置知识：最大独立集
1. 图论定义：给定无向图$G=(V,E)$，独立集中任意两个节点之间没有边相连，极大独立集是一个独立集，且无法再加入任何节点而保持独立
2. 分布式MIS问题：每个节点是一个进程，只知道自己的邻居，目标是让每个节点最终决定IN或OUT（是否在MIS中）

模型假设：同步模型，全局轮次，随机数独立生成，通信可靠

算法流程：
1. 初始化：$status := UNDECIDED$
2. 在每一轮中：

$\ \ \ \ \ \ $ step1.随机竞争：每个$UNDECIDED$节点$v$生成随机数$p_v~U(0,1)$，向所有邻居发送$p_v$

$\ \ \ \ \ \ $ step2.局部比较：对于节点$v$，若$p_v\geq p_u \ \forall u \in N(v)$，则$status := IN$

$\ \ \ \ \ \ $ step3.邻居退出：所有与$IN$节点相邻的$UNDECIDED$节点：$status := OUT$

$\ \ \ \ \ \ $ step4.删除已决定节点：$IN$和$OUT$节点从图中移除，剩余节点进入下一轮
3. 终止条件：所有节点状态$!=UNDECIDED$

时间复杂度：$O(logn)$ $\ \ \ $消息复杂度：$O(Elogn)$

In [2]:
import random
UNDECIDED="UNDECIDED"
IN="IN"
OUT="OUT"

class Node:
    def __init__(self, node_id):
        self.id = node_id
        self.neighbors = set()
        self.status = UNDECIDED
        self.priority = None

    def __repr__(self):
        return f"Node({self.id}, {self.status})"

class LubyMIS:
    def __init__(self, graph):
        """
        graph: dict {node_id: [neighbor_ids]}
        """
        self.nodes={i: Node(i) for i in graph}
        for i, nbrs in graph.items():
            self.nodes[i].neighbors=set(nbrs)
        self.round=0

    def active_nodes(self):
        return [
            n for n in self.nodes.values()
            if n.status == UNDECIDED
        ]

    def run(self, verbose=True):
        while self.active_nodes():
            self.round+=1
            if verbose:
                print(f"\n=== Round {self.round} ===")
            active = self.active_nodes()
            # Step 1: generate random priorities
            for node in active:
                node.priority=random.random()
                if verbose:
                    print(
                        f"Node {node.id} priority = "
                        f"{node.priority:.4f}"
                    )
            # Step 2: decide IN nodes
            new_in=[]
            for node in active:
                higher_than_all=True
                for nbr_id in node.neighbors:
                    nbr=self.nodes[nbr_id]
                    if nbr.status==UNDECIDED:
                        if nbr.priority>=node.priority:
                            higher_than_all=False
                            break
                if higher_than_all:
                    new_in.append(node)
            #Step 3: mark IN and OUT
            for node in new_in:
                node.status=IN
                if verbose:
                    print(f"Node {node.id} enters MIS")
            for node in new_in:
                for nbr_id in node.neighbors:
                    nbr=self.nodes[nbr_id]
                    if nbr.status==UNDECIDED:
                        nbr.status=OUT
                        if verbose:
                            print(
                                f"Node {nbr.id} "
                                f"excluded by {node.id}"
                            )
            # Step 4: clear priorities
            for node in self.nodes.values():
                node.priority=None

        if verbose:
            print("\nAlgorithm finished.")

    def result(self):
        return {
            node.id: node.status
            for node in self.nodes.values()
        }

    def mis_nodes(self):
        return [
            node.id for node in self.nodes.values()
            if node.status == IN
        ]

if __name__ == "__main__":
    graph = {
        1: [2, 3],
        2: [1, 3, 4],
        3: [1, 2, 4],
        4: [2, 3, 5],
        5: [4]
    }

    sim = LubyMIS(graph)
    sim.run(verbose=True)

    print("\nFinal MIS:", sim.mis_nodes())



=== Round 1 ===
Node 1 priority = 0.7916
Node 2 priority = 0.3894
Node 3 priority = 0.2650
Node 4 priority = 0.9706
Node 5 priority = 0.9378
Node 1 enters MIS
Node 4 enters MIS
Node 2 excluded by 1
Node 3 excluded by 1
Node 5 excluded by 4

Algorithm finished.

Final MIS: [1, 4]
